# Imports

In [128]:
import sys 
sys.path.append('..')

from itertools import chain

from util.nos_articles import *
import pandas as pd
from tqdm.notebook import tqdm
import spacy
# nlp = spacy.load("nl_core_news_sm")
nlp = spacy.load("nl_core_news_lg")

# Download Articles

In [129]:
dates = [x.strftime('%Y-%m-%d') for x in pd.date_range(start='2020-04-01', end='2020-12-31', freq='D')]

In [130]:
# download_articles(dates=dates, max_articles=5000, refresh_articles=False)

# Load Articles

In [131]:
articles_dict = load_nos_texts(dates=dates)

  0%|          | 0/275 [00:00<?, ?it/s]

In [ ]:
# articles_dict['2020-04-01']

# Prepare Data

In [177]:
list_of_articles = list(chain(*articles_dict.values()))
list_of_articles = [' '.join(x) for x in list_of_articles]
print(len(list_of_articles))
print(list_of_articles[1])

print(len(chunks))
list_of_text = [item for sublist in list_of_articles for item in sublist]
print(len(list_of_text))
chunk_size =  int(len(list_of_text)/500)
print(chunk_size)
chunks = [list_of_text[x:x+chunk_size] for x in range(0, len(list_of_text), chunk_size)]
#chunks[1]
string_chunks = [' '.join(x) for x in chunks]
#len(list_of_text)
#stringed_text = ' '.join(list_of_text)
#stringed_text[:500]
len(string_chunks)

4990
De biograaf van de Britse prins Harry heeft in een artikel in modemagazine Tatler hard uitgehaald naar Harry en zijn vrouw Meghan, die sinds gisteren geen volwaardige leden van het Britse koningshuis meer zijn.
 Schrijfster Angela Levin, die voor haar boek een jaar mocht optrekken met Harry, leerde de prins in 2017 kennen als een charismatische en grappige man.
 Drie jaar later ziet ze "een enorme verandering in zijn doen en laten". Volgens Levin zijn Harry en Meghan "toenemend egoïstisch", doelend op het besluit van het paar een stap terug te doen in het Britse koningshuis en te verhuizen naar Noord-Amerika. "Harry is bitter geworden en komt continu gestresst over", aldus Levin.
 Volgens haar had de Harry die ze zo goed leerde kennen nooit zijn familie de rug toegekeerd. En al helemaal niet temidden van "een nationale crisis", schrijft ze, refererend aan de coronacrisis en het gegeven dat Harry's vader kroonprins Charles besmet is met het virus. Ze herkent de prins naar eigen zeg

501

# Calculate Average Sentence Length

In [46]:
average_sentence_length = get_average_sentence_length(stringed_text)
average_sentence_length

17.202796550050405

In [191]:
TF_IDF_CSV_PATH = '../output/NOS_tf_idf.csv'
TF_CSV_PATH = '../output/NOS_tf.csv'
CORPUS_PATH = '../output/NOS_corpus_5000_articles_without_stopwords.csv'

# Create Corpus

In [180]:
def create_corpus(full_text: list, save=False, stopwords=False) -> tuple:
    nlp.max_length = 1500000
    processed_texts = [text for text in tqdm(nlp.pipe(full_text), total=len(full_text))]
    
    lemmatized_texts = [[word.lemma_ for word in processed_text if not (word.is_punct or word.is_space or (stopwords and word.is_stop))] \
                        for processed_text in tqdm(processed_texts, total=len(processed_texts))]

    flattened_lemmatized_texts = [item for sublist in lemmatized_texts for item in sublist]
    word_count = Counter(flattened_lemmatized_texts)
    
    # Create Dataframe with Word Counts
    corpus = pd.DataFrame(columns=['word'])
    
    # Allows you to differentiate between subsets by creating columns for each of them
    new_word = list(set(word_count.keys()) - set(corpus['word']))
    corpus = corpus.append(pd.DataFrame({'word': new_word}), ignore_index=True)

    wordlist = []
    for word in corpus['word']:
        if word in word_count.keys():
            wordlist.append(word_count[word])
        else:
            wordlist.append(0)

    corpus['nos'] = wordlist
    corpus.set_index('word', inplace=True)
    corpus.fillna(0, inplace=True)
    
    if save:
        corpus.to_csv(CORPUS_PATH)
    
    return corpus

In [160]:
string_chunks[1][:500].split(' ')

['Daarom',
 'zijn',
 'scholen',
 'gewoon',
 'open,',
 'zodat',
 'ouders',
 'van',
 'kinderen',
 'met',
 'cruciale',
 'beroepen',
 'kunnen',
 'blijven',
 'werken.',
 'Ook',
 'is',
 'er',
 'volgens',
 'de',
 'Zweedse',
 'wetenschappers',
 'onvoldoende',
 'bewijs',
 'dat',
 'het',
 'coronavirus',
 'zich',
 'sneller',
 'via',
 'kinderen',
 'op',
 'scholen',
 'verspreidt.\n',
 'Alleen',
 'voor',
 'ouderen',
 'gelden',
 'er',
 'voorlopig',
 'extra',
 'maatregelen.',
 'Zo',
 'worden',
 'mensen',
 'boven',
 'de',
 '70',
 'aangeraden',
 'om',
 'binnen',
 'te',
 'blijven',
 'en',
 'is',
 'ook',
 'in',
 'Zweedse',
 'verpleeghuizen',
 'voorlopig',
 'geen',
 'bezoek',
 'toegestaan.\n',
 'Dat',
 'er',
 'maar',
 'weinig',
 'onrust',
 'is',
 'in',
 'Zweden',
 'over',
 'het',
 'losse',
 'coronabeleid',
 'komt',
 '']

In [184]:
# string_chunks[1][:5000].split(' ')
%time
corpus = create_corpus(list_of_articles, save=False)
corpus

Wall time: 0 ns


  0%|          | 0/4990 [00:00<?, ?it/s]

  0%|          | 0/4990 [00:00<?, ?it/s]

,nos
word,
jeugdig,1
winnen,187
katinka,2
veiligheidsbril,2
tijdstraf,1
...,...
wvggz,1
ryanair,26
werk-voucher,1


In [192]:
corpus.sort_values(by='nos', ascending=False)

,nos
word,
gaan,8382
zeggen,8039
komen,7166
mens,6885
jaar,5897
...,...
politiekleding,1
barak,1
kerkstraat,1


In [190]:
corpus['nos'].sum()

855629

In [193]:
corpus.to_csv(CORPUS_PATH)

# Create tf & tf-idf

In [21]:
def calculate_tfidf(corpus, include_idf=True):
    # Create Dataframe with Relative Word Frequencies
    tf_idf = {k: [] for k in corpus.columns}
    for index, row in tqdm(corpus.iterrows(), total=len(corpus)):
        docs_with = np.count_nonzero(row)

        for colname, count in row.items():
            total_uniques = np.count_nonzero(corpus[colname])
            tf = count / total_uniques
            result = tf
            if include_idf:
                idf = math.log(len(corpus.columns) / docs_with)
                result *= idf

            tf_idf[colname].append(result)

    tf_idf_df = pd.DataFrame.from_dict(tf_idf)
    tf_idf_df.set_index(corpus.index, inplace=True)

    if include_idf:
        tf_idf_df.to_csv(TF_IDF_CSV_PATH)
        print(f"Saved results to 'output/corpus.csv and '{TF_IDF_CSV_PATH}'")
    else:
        tf_idf_df.to_csv(TF_CSV_PATH)
        print(f"Saved results to 'output/corpus.csv' and '{TF_CSV_PATH}'")

In [ ]:
calculate_tfidf(full_text=stringified_text, include_idf=True):

# Basis woorden